In [ ]:
!nvidia-smi

Fri Jan 29 11:11:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will print the CUDA version of the runtime if it has a GPU, and install PyTorch 1.7.1.

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

# torch_version_suffix = '+cpu'   

CUDA version: 10.1


In [ ]:
! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4MB 24kB/s 
     |████████████████████████████████| 12.8MB 244kB/s 
     |████████████████████████████████| 71kB 3.9MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=8e22735b99837255b93f1f0641694c69373e013e9ca9aeb8f9e13c8c64f5b1b3
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [ ]:
import numpy as np
import torch

print("Torch version:", torch.__version__)

Torch version: 1.7.0+cu101


Mounted at /content/drive


In [ ]:
### Check if Shared folder is available :
"""
You need to create shorcut of this SHARED folder

https://drive.google.com/drive/u/0/folders/1nS-yYgKNmkrelX9Z82BvjrGClI-Ar4Hp


IN your Google Drive

    /MyDrive/Colab Notebooks/shared_session/


! ls "/content/drive/MyDrive/Colab Notebooks/shared_session/"


"""


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##### Go to root folder
root =   "/content/drive/My Drive/Colab Notebooks/shared_session/img/"

import os, sys
print(os.getcwd())
os.chdir(root)
print( os.getcwd())

! pwd && ls .




/content


FileNotFoundError: ignored

# Kaggle

In [ ]:
"""
https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a

import os
os.environ['KAGGLE_CONFIG_DIR'] = root
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive



"""

import os
os.environ['KAGGLE_CONFIG_DIR'] = root

print(  os.environ['KAGGLE_CONFIG_DIR'] )

/content/drive/My Drive/Colab Notebooks/shared_session/img/


In [ ]:


! kaggle datasets download -d paramaggarwal/fashion-product-images-dataset

100% 23.1G/23.1G [10:08<00:00, 53.3MB/s]
100% 23.1G/23.1G [10:08<00:00, 40.7MB/s]


In [ ]:
! ls -l

total 24190663
drwx------ 2 root root        4096 Jan 14 11:11 data
-rw------- 1 root root 24771215740 Jan 14 11:41 fashion-product-images-dataset.zip
-rw------- 1 root root          63 Jan 14 11:27 kaggle.json
-rw------- 1 root root       13839 Jan 14 11:43 transfer.ipynb
drwx------ 2 root root        4096 Jan 14 11:11 zperso


In [ ]:
! pwd

/content/drive/My Drive/Colab Notebooks/shared_session/img


In [ ]:
! mv  fashion-product-images-dataset.zip  data/

In [ ]:
! cd data &&  nohup unzip fashion-product-images-dataset.zip &



nohup: appending output to 'nohup.out'


# Downloading the model
TorchScript modules.

In [ ]:

##### Go to root folder
root =   "/content/drive/My Drive/Colab Notebooks/shared_session/img/"

import os, sys
print( os.getcwd())
os.chdir(root)
print( os.getcwd())

! pwd && ls .






/content


FileNotFoundError: ignored

# Image Preprocessing

We resize the input images and center-crop them to conform with the image resolution that the model expects. Before doing so, we will normalize the pixel intensity using the dataset mean and standard deviation.



In [ ]:
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image

preprocess = Compose([
    Resize(input_resolution, interpolation=Image.BICUBIC),
    CenterCrop(input_resolution),
    ToTensor()
])

image_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
image_std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()

In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



# Training Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tez
!pip install efficientnet_pytorch
!pip uninstall albumentations
!pip install albumentations

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16032 sha256=b762b442ba6c90b0605c5700676f2ce1e28d9c9188def8a6b74592e3dc69e143
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
Uninstalling albumentations-0.1.12:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/albumentations-0.1.12.dist-info/*
    /usr/local/lib/python3.6/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 952kB 8.0MB/s 
     |████████████████████████████████| 37.6MB 85kB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
# import os,sys
# !pip install tez -y
# !pip install efficientnet_pytorch
# !pip uninstall albumentations
# !pip install albumentations


In [ ]:
root = '/content/drive/MyDrive/upwork/img'
# root =   "/content/drive/My Drive/Colab Notebooks/shared_session/img/"


import os, sys
print( os.getcwd())
os.chdir(root)
print( os.getcwd())


/content
/content/drive/.shortcut-targets-by-id/1nS-yYgKNmkrelX9Z82BvjrGClI-Ar4Hp/img


In [ ]:
"""

! nohup  python3 model_transfer.py  &

!python3 model_transfer.py

!python3 extract_vectors_big_model.py


"""



In [ ]:
print(len(os.listdir('/content/drive/MyDrive/upwork/img/dataset/images')))

OSError: ignored

In [ ]:
!ls

 data					   model_transfer_v2.py
 dataset				   model_transfer_v3.py
 df_final.csv				   multi_output_classifier.ipynb
 df_final_pred.csv			   nohup.out
 df_final_with_original_class.csv	   pred_df.csv
 extract_vectors_big_model.py		   styles.csv
 fashion-dataset			   test.csv
 fashion_dataset_feature_extractor.ipynb   test_pred.csv
 kaggle.json				  'test_pred.csv--batch_size=32'
 model1_new				   train.csv
 models					   transfer.ipynb
 model_transfer.py			   val.csv


In [ ]:
# !unzip './data/fashion-product-images-dataset.zip' -d './data/'

In [ ]:
import time
while True:
  time.sleep(10)
  pass

KeyboardInterrupt: ignored

In [ ]:
len(os.listdir('./dataset/images'))

44441

## Stage 1 : DF Preprocess

In [ ]:
!python model_transfer_v3.py --stage='df_preprocess' --base_path='./' --image_path='./dataset/images' \
                            --base_df='styles.csv' --root_df='df_final.csv' --train_df='train.csv' --val_df='val.csv'




df_preprocess
Total Number of Images:  44441
b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expect

## Stage 2 : Extract Efficient

In [ ]:
!python -V

Python 3.6.9


In [ ]:
# !python model_transfer_v2.py --stage='extract_efficienet' --base_path='./' --image_path='./data/fashion-dataset/images' \
#                              --output_vector_path='./data/fashion-dataset/efficienet_vectors'



# !python model_transfer_v3.py --stage='extract_efficienet' --base_path='./' --image_path='./images' \
#                             --output_vector_path='./data/fashion-dataset/efficientnet_vectors' \
#                             --root_df='df_final.csv' --test_df='df_final.csv' --batch_size=32

!nohup python3 model_transfer_v3.py --stage='extract_efficienet' --base_path='./' --image_path='./dataset/images' \
                            --output_vector_path='./dataset/efficientnet_vectors' \
                            --root_df='df_final.csv' --test_df='df_final.csv' --batch_size=16 &

nohup: appending output to 'nohup.out'


In [ ]:
len(os.listdir('/content/drive/MyDrive/upwork/img/dataset/efficientnet_vectors'))
# len(os.listdir('/content/drive/MyDrive/upwork/img/data/fashion-dataset/efficientnet_vectors'))

0

## Stage 2 : Extract Big Model1


In [ ]:
# !python model_transfer_v2.py --stage='extract_bigmodel1' --base_path='./' --image_path='./data/fashion-dataset/images' \
#                        --output_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='model'


# ! python model_transfer_v2.py --stage='extract_bigmodel1' --base_path='./' --image_path='./data/fashion-dataset/images' \
#                       --output_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='model'


# !python model_transfer_v3.py --stage='extract_bigmodel1' --base_path='./' --image_path='./data/fashion-dataset/images' \
#                             --output_vector_path='./fashion-dataset/big_model1_vectors' --model_name='model' \
#                             --root_df='df_final.csv' --test_df='df_final.csv' --batch_size=32

!nohup python3 model_transfer_v3.py --stage='extract_bigmodel1' --base_path='./' --image_path='./images' \
                            --output_vector_path='./fashion-dataset/big_model1_vectors' --model_name='model' \
                            --root_df='df_final.csv' --test_df='df_final.csv' --batch_size=32 &


nohup: appending output to 'nohup.out'


## Stage 3 : train from big model1 vectors

In [ ]:
# # !python model_transfer_v2.py --stage='train_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
# #         --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='fashion_big_model1' \
# #         --input_vector_size=512 --intermediate_vector_size=128


!python model_transfer_v3.py --stage='train_small' --base_path='./' --image_path='./images' \
        --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='fashion_big_model1' \
        --input_vector_size=512 --intermediate_vector_size=128 --root_df='df_final.csv' --train_df='train.csv' --val_df='val.csv' --batch_size=32 --epochs=20 --save_interval=5


# !nohup python3 model_transfer_v3.py --stage='train_small' --base_path='./' --image_path='./images' \
#         --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='fashion_big_model1' \
#         --input_vector_size=512 --intermediate_vector_size=128 --root_df='df_final.csv' --train_df='train.csv' \
#         --val_df='val.csv' --batch_size=32 --epochs=20 --save_interval=5 &
        

train_small
Total Number of Images:  44441
Starting  Train Small model -------------------------
  0% 0/1098 [00:00<?, ?it/s]Traceback (most recent call last):
  File "model_transfer_v3.py", line 532, in <module>
    fp16          = args.fp16,
  File "/usr/local/lib/python3.6/dist-packages/tez/model/model.py", line 324, in fit
    train_loss = self.train_one_epoch(self.train_loader)
  File "/usr/local/lib/python3.6/dist-packages/tez/model/model.py", line 193, in train_one_epoch
    for b_idx, data in enumerate(tk0):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1085, in _next_data
    return self._process_data(data)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1111, in _pr

## Stage 4 : train from efficient vectors

In [ ]:
# !ls ./data/fashion-dataset/efficientnet_vectors

In [ ]:
# !python model_transfer_v2.py --stage='train_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
#               --intermediate_vector_path='./data/fashion-dataset/efficient_vectors' --model_name='fashion_efficient' \
#               --input_vector_size=1536 --intermediate_vector_size=512  --epochs 1

              


!python model_transfer_v3.py --stage='train_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
          --intermediate_vector_path='./data/fashion-dataset/efficientnet_vectors' --model_name='fashion_efficient' \
          --input_vector_size=1536 --intermediate_vector_size=512 --root_df='df_final.csv' --train_df='train.csv' --val_df='val.csv' --batch_size=32 --epochs=20 --save_interval=5



## Stage 5 : extract final vectors

In [ ]:
# !ls ./models

In [ ]:
# !python model_transfer_v2.py --stage='extract_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
#       --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --output_vector_path='./data/fashion-dataset/final_vectors' \
#       --model_name='fashion_big_model1' --input_vector_size=512 --intermediate_vector_size=128  --epochs  1


#! nohup python model_transfer_v2.py --stage='extract_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
#      --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --output_vector_path='./data/fashion-dataset/final_vectors' \
#      --model_name='fashion_big_model1' --input_vector_size=512 --intermediate_vector_size=128  --epochs  1   &


            
!python model_transfer_v3.py --stage='extract_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
      --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors'  --output_vector_path='./data/fashion-dataset/final_vectors' \
      --model_name='fashion_big_model1_best' --input_vector_size=512 --intermediate_vector_size=128 --root_df='df_final.csv' --test_df='df_final.csv'



## Stage 5 : train from logistic regression

In [ ]:
# ! python model_transfer_v2.py --stage='check_vectors' --base_path='./' --image_path='./data/fashion-dataset/images' \
#          --intermediate_vector_path='./data/fashion-dataset/final_vectors' \
#          --model_name='logistic' --logistic_reg_targ_col='gender'


!python model_transfer_v3.py --stage='check_vectors' --base_path='./' --image_path='./data/fashion-dataset/images' \
        --intermediate_vector_path='./data/fashion-dataset/final_vectors' --model_name='logistic' \
        --logistic_reg_targ_col='gender' --root_df='df_final.csv' --train_df='train.csv' --val_df='val.csv' 


## Inference

## Stage 6: Test :> Extract Big Model1

In [ ]:
!python model_transfer_v3.py --stage='extract_bigmodel1' --base_path='./' --image_path='./data/test_dataset/images' \
        --output_vector_path='./data/test_dataset/big_model1_vectors' --model_name='model' --root_df='df_final.csv' --test_df='test.csv' --batch_size=32

extract_bigmodel1
Total Number of Images:  6760
Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408
100% 212/212 [25:52<00:00,  7.32s/it]
Stage extract_bigmodel1 Finished ------------------------------------------------


In [ ]:
# while True:
#   pass

## Stage 7: Test :> extract final vectors

In [ ]:
!python model_transfer_v3.py --stage='extract_small' --base_path='./' --image_path='./data/test_dataset/images' \
          --intermediate_vector_path='./data/test_dataset/big_model1_vectors'  --output_vector_path='./data/test_dataset/final_vectors' \
          --model_name='fashion_big_model1_best' --input_vector_size=512 --intermediate_vector_size=128 --root_df='df_final.csv' --test_df='test.csv' --batch_size=32

extract_small
Total Number of Images:  6760
Starting Extract Vector from Small model -------------------------
100% 212/212 [25:03<00:00,  7.09s/it]
Stage: extract_small Finished -------------------------


## Stage 8 : Test :> prediction

**This will generate pred_df.csv and if label are present in input df then it will also calculate accuracy**

In [ ]:
# Prdiction on labeled data
!python model_transfer_v3.py --stage='predict_small' --base_path='./' --image_path='./data/fashion-dataset/images' \
        --intermediate_vector_path='./data/fashion-dataset/big_model1_vectors' --model_name='fashion_big_model1_best' --input_vector_size=512 \
        --intermediate_vector_size=128 --root_df='df_final.csv' --test_df='df_final_with_original_class.csv' --base_df='df_final_with_original_class.csv' \
        --pred_df='df_final_pred.csv' --batch_size=32

predict_small
Total Number of Images:  29163
Starting Extract Vector from Small model -------------------------
100% 905/905 [00:23<00:00, 37.72it/s]
(28946, 8)
Class Name: gender ========== Accuracy: 0.917
Class Name: masterCategory ========== Accuracy: 0.995
Class Name: subCategory ========== Accuracy: 0.972
Class Name: articleType ========== Accuracy: 0.900
Class Name: baseColour ========== Accuracy: 0.709
Class Name: season ========== Accuracy: 0.708
Class Name: usage ========== Accuracy: 0.926
Prediction file saved to  df_final_pred.csv
Stage: predict_small Finished ------------------------------------------------


In [ ]:
# Prdiction on unlabeled data

!python model_transfer_v3.py --stage='predict_small' --base_path='./' --image_path='./data/test_dataset/images' \
        --intermediate_vector_path='./data/test_dataset/big_model1_vectors' --model_name='fashion_big_model1_best' --input_vector_size=512 \
        --intermediate_vector_size=128 --root_df='df_final.csv' --test_df='test.csv' --base_df='df_final_with_original_class.csv' \
        --batch_size=32 --pred_df='test_pred.csv'

predict_small
Total Number of Images:  6760
Starting Extract Vector from Small model -------------------------
100% 212/212 [00:05<00:00, 36.23it/s]
(6760, 8)
Prediction file saved to  test_pred.csv
Stage: predict_small Finished ------------------------------------------------


# Batch

In [ ]:
root =   "/content/drive/My Drive/Colab Notebooks/shared_session/img/"

!pip install tez -y
!pip install efficientnet_pytorch
!pip uninstall albumentations
!pip install albumentations

Uninstalling albumentations-0.1.12:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/albumentations-0.1.12.dist-info/*
    /usr/local/lib/python3.6/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 952kB 12.9MB/s 
     |████████████████████████████████| 37.6MB 82kB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
#### Extract from Big Model

!python3 extract_vectors_big_model.py  --

In [ ]:
#### Extract big model vector
!  python3 model_transfer.py  --stage extract_bigmodel1   --intermediate_vector_path  './data/fashion-dataset/bigmodel_vectors/' 





In [ ]:
#### Train Small model

!  python3 model_transfer.py  --stage train_small   --intermediate_vector_path  './data/fashion-dataset/bigmodel_vectors/' \
        --epochs 1  








"""

!  nohup python3 model_transfer.py  --stage train_small   --intermediate_vector_path  './data/fashion-dataset/bigmodel_vectors/' \
        --epochs 1   &
"""



train_small
Starting  Train Small model -------------------------
  0% 0/2895 [00:00<?, ?it/s]Traceback (most recent call last):
  File "model_transfer.py", line 360, in <module>
    fp16          = args.fp16,
  File "/usr/local/lib/python3.6/dist-packages/tez/model/model.py", line 297, in fit
    train_loss = self.train_one_epoch(self.train_loader, device)
  File "/usr/local/lib/python3.6/dist-packages/tez/model/model.py", line 178, in train_one_epoch
    for b_idx, data in enumerate(tk0):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1085, in _next_data
    return self._process_data(data)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1111, in _process_data
    data.rerais

"\n\n!  nohup python3 model_transfer.py  --stage train_small   --intermediate_vector_path  './data/fashion-dataset/bigmodel_vectors/'         --epochs 1   &\n"

In [ ]:
# ! ls   './data/fashion-dataset/bigmodel_vectors/'    | wc -l


! ls   ./data/fashion-dataset/bigmodel_vectors/*1404*  




./data/fashion-dataset/bigmodel_vectors/14040.npy
./data/fashion-dataset/bigmodel_vectors/14041.npy
./data/fashion-dataset/bigmodel_vectors/14047.npy
./data/fashion-dataset/bigmodel_vectors/14048.npy
./data/fashion-dataset/bigmodel_vectors/14049.npy
./data/fashion-dataset/bigmodel_vectors/41404.npy


In [ ]:
#### Extract from Small Model



In [ ]:
! pwd

/content/drive/My Drive/Colab Notebooks/shared_session/img


In [ ]:
#### Test from small model vectors and output accuracy

!  python3 model_transfer.py  --stage check_vectors   --intermediate_vector_path  './data/fashion-dataset/final_vectors_v01/'




# !  nohup python3 model_transfer.py  --stage check_vectors   --imtermediate_vector_path  './data/fashion-dataset/final_vectors_v01/' \
#          &


check_vectors
Starting   ------------------------- check_vectors
Traceback (most recent call last):
  File "model_transfer.py", line 397, in <module>
    train_dataset = FashionImageDataset(IMAGE_PATH=INTERMEDIATE_VECTOR_PATH, DF_PATH=BASE_DIR/'train.csv')
  File "model_transfer.py", line 57, in __init__
    self.df              = pd.read_csv(DF_PATH)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 688, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 454, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 948, in __init__
    self._make_engine(self.engine)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1180, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 2010, in __init__
